# Tableau Credential Management

#### This script can help manage credentials programatically. It includes 4 components: 
1. Add new users to multiple sites;
2. Change existing users' password;
3. Get user info from multiple sites;
4. Remove users from multiple sites.

## ***Add New Users

In [115]:
from tableau_rest_api.tableau_rest_api import *
import csv

server = 'https://bi.tune.com'
loginName = 'xiaojuan'
loginPasswd = 'Has0ffers'
user_info = []

def read_csv_file(fileName):
    with open(fileName, 'rU') as file:
        read_file = csv.reader(file)
        header = read_file.next()
        for line in read_file:
            user_info.append(line)
    file.close()
    return user_info

def add_user(user_info):
    for site, username, fullname, siterole, password, email in user_info:
        t = TableauRestApi(server, loginName, loginPasswd, site)
        t.signin()
        #print 'Signin Succeed'
        t.add_user_by_username(username, site_role = siterole, update_if_exists=True)
        print '%s added to Site: %s'%(username, site)
    t.signout()
    print 'All Users Added'
    for site, username, fullname, siterole, password, email in user_info:
        t = TableauRestApi(server, loginName, loginPasswd, site)
        t.signin()
        users_in_server = t.query_users()
        for user in users_in_server:
            if user.get('name') == username:
                t.update_user_by_username(user.get('name'), password = password, full_name = fullname, email = email)
                print 'User: %s updated'%username
    t.signout()
    print 'All user updated'
    print 'Mission completed'

In [116]:
fileName = 'add_user.csv'
user_info = read_csv_file(fileName)
add_user(user_info)

test1 added to Site: TuneBIDemo
test2 added to Site: Uber
test3 added to Site: BetaTest
test4 added to Site: BetaTest2
All Users Added
User: test1 updated
User: test2 updated
User: test3 updated
User: test4 updated
All user updated
Mission completed


## *Update User Password

In [120]:
from tableau_rest_api.tableau_rest_api import *
import csv

server = 'https://bi.tune.com'
loginname = 'xiaojuan'
loginPass = 'Has0ffers'
user_info = []

def read_csv_file(fileName):
    with open(fileName, 'rU') as file:
        read_file = csv.reader(file)
        header = read_file.next()
        for line in read_file:
            user_info.append(line)
    file.close()
    return user_info

def update_passwd(userInfo):   
    for site, user, passwd in userInfo:
        t = TableauRestApi(server, loginname, loginPass, site)
        t.signin()
        t.update_user(user, password = passwd)
        print '%s Password Updated from %s'%(user, site)
    t.signout()
    print 'Mission Completed!!!'

In [121]:
fileName = 'ChangePassword.csv'
userInfo = read_csv_file(fileName)
update_passwd(userInfo)

test1 Password Updated from TuneBIDemo
test2 Password Updated from Uber
test3 Password Updated from BetaTest
test4 Password Updated from BetaTest2
Mission Completed!!!


## *Get User Info and Generate CSV File

In [129]:
from tableau_rest_api.tableau_rest_api import *
import pandas as pd
import csv

server = 'https://bi.tune.com'
loginName = 'xiaojuan'
loginPasswd = 'Has0ffers'

def read_csv_file(fileName):
    site_list = []
    with open(fileName, 'rU') as file:
        read_file = csv.reader(file)
        #header = read_file.next()
        for line in read_file:
            site_list.append(line)
    file.close()
    site_list = [i for site in site_list for i in site]
    return site_list

def user_info(site_list):
    username = []
    siterole = []
    lastLogin = []
    newSite = []
    valid_site_role = ['Guest', 'ServerAdministrator']
    for site in site_list:
        t = TableauRestApi(server, loginName, loginPasswd, site)
        t.signin()
        #print '%s Signin Succeed'%site
        for user in t.query_users():
            if user.get('siteRole') not in valid_site_role:
                username.append(user.get('name'))
                lastLogin.append(user.get('lastLogin'))
                siterole.append(user.get('siteRole'))
                newSite.append(site)
        print 'Get the User info from %s'%site
    #print username, siterole, lastLogin
    df_user = pd.DataFrame({'site':newSite, 'username':username, 'siteRole':siterole, 'lastLogin':lastLogin})
    df_user.to_csv('user.csv', sep = ',')
    t.signout()
    print 'CSV file generated'

In [130]:
site = read_csv_file('siteList.csv')
user_info(site)

Get the User info from TuneBIDemo
Get the User info from BetaTest
Get the User info from BetaTest2
Get the User info from Uber
Get the User info from 433
CSV file generated


## *Remove Users

In [132]:
from tableau_rest_api.tableau_rest_api import *
import csv

server = 'https://bi.tune.com'
loginName = 'xiaojuan'
loginPasswd = 'Has0ffers'
user_info = []

def read_csv_file(fileName):
    with open(fileName, 'rU') as file:
        read_file = csv.reader(file)
        header = read_file.next()
        for line in read_file:
            user_info.append(line)
    file.close()
    return user_info

def remove_users(userInfo):
    for site, username in userInfo:
        t = TableauRestApi(server, loginname, loginPass, site)
        t.signin()
        users_in_server = t.query_users()
        for user in users_in_server:
            if user.get('name') != username:
                continue
            else:
                userId = user.get('id')
                t.remove_users_from_site_by_luid(userId)
                print '%s Removed from %s'%(username, site)
    t.signout()
    print 'Removed all users'
    print 'Mission Completed!!!'

In [133]:
userInfo = read_csv_file('remove_user.csv')
remove_users(userInfo)

test1 Removed from TuneBIDemo
test2 Removed from Uber
test3 Removed from BetaTest
test4 Removed from BetaTest2
Removed all users
Mission Completed!!!
